### Setup

In [120]:
import tensorflow.compat.v2  as tf
tf.enable_v2_behavior()

### Time windows

In [121]:
dataset = tf.data.Dataset.range(10)
for data in dataset:
    print(data.numpy(), end = "\t")
    print(data)

0	tf.Tensor(0, shape=(), dtype=int64)
1	tf.Tensor(1, shape=(), dtype=int64)
2	tf.Tensor(2, shape=(), dtype=int64)
3	tf.Tensor(3, shape=(), dtype=int64)
4	tf.Tensor(4, shape=(), dtype=int64)
5	tf.Tensor(5, shape=(), dtype=int64)
6	tf.Tensor(6, shape=(), dtype=int64)
7	tf.Tensor(7, shape=(), dtype=int64)
8	tf.Tensor(8, shape=(), dtype=int64)
9	tf.Tensor(9, shape=(), dtype=int64)


In [122]:
dataset = tf.data.Dataset.range(10)
# dataset = dataset.window(3, shift = 2)
dataset = dataset.window(5, shift = 1)

for window_data in dataset:
    for val in window_data:
        print(val.numpy(), end = " ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 
6 7 8 9 
7 8 9 
8 9 
9 


2024-03-02 11:38:10.687739: W tensorflow/core/framework/dataset.cc:956] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


In [123]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift = 1, drop_remainder = True)
for dataset_window in dataset:
    for data in dataset_window:
        print(data.numpy(), end = " ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 


In [124]:
dataset = tf.data.Dataset.range(10)
# dataset = dataset.window(4, shift = 1, drop_remainder = True)
# dataset = dataset.flat_map(lambda dataset_window: dataset_window.batch(4))
dataset = dataset.window(5, shift = 1, drop_remainder = True).flat_map(lambda window: window.batch(5))

for window in dataset:
    print(window.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


In [125]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift = 1, drop_remainder = True).flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))

for x, y in dataset:
    print(x.numpy(), y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


In [126]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift = 1, drop_remainder = True).flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:])).shuffle(buffer_size = 10)

for x, y  in dataset:
    print(x.numpy(), y.numpy())

[2 3 4 5] [6]
[5 6 7 8] [9]
[4 5 6 7] [8]
[1 2 3 4] [5]
[0 1 2 3] [4]
[3 4 5 6] [7]


In [127]:
dataset = tf.data.Dataset.range(10).window(5, shift = 1, drop_remainder = True).flat_map(
    lambda window: window.batch(5))

dataset = dataset.map(lambda window: (window[:-1], window[-1:])).shuffle(buffer_size = 10).batch(2).prefetch(1)

for x, y in dataset:
    print("x = ",x.numpy())
    print("y = ",y.numpy())

x =  [[1 2 3 4]
 [4 5 6 7]]
y =  [[5]
 [8]]
x =  [[2 3 4 5]
 [3 4 5 6]]
y =  [[6]
 [7]]
x =  [[5 6 7 8]
 [0 1 2 3]]
y =  [[9]
 [4]]


In [128]:
def window_dataset(series, window_size, batch_size = 32, buffer_size = 1000):
    # load dataset from a time series
    dataset = tf.data.Dataset.from_tensor_slices(series)
    # create a window from the dataset
    dataset = dataset.window(window_size + 1, shift = 1, drop_remainder = True)
    # let the windows appear as []
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    # shuffle the dataset
    dataset = dataset.shuffle(buffer_size)
    # map x and y labels
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    # load the data as a batch and prefetch to ensure the next data is loading. GPU is never idle.
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [129]:
import numpy as np
series = np.array(range(10))
for x, y in window_dataset(series, 4, 2, 10):
    print("x = ", x.numpy())
    print("y = ", y.numpy())

x =  [[0 1 2 3]
 [1 2 3 4]]
y =  [4 5]
x =  [[4 5 6 7]
 [3 4 5 6]]
y =  [8 7]
x =  [[5 6 7 8]
 [2 3 4 5]]
y =  [9 6]
